In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Dropout, MultiHeadAttention, Add, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
site_files = ["site_1_train_data.csv", "site_2_train_data.csv", "site_3_train_data.csv"]

dfs = {}
train_dfs = {}
test_dfs = {}
scaler_xs = {}
scaler_ys = {}
X_train_seqs = {}
y_train_seqs = {}
X_test_seqs = {}
y_test_seqs = {}
results = {}
time_steps = 72

In [ ]:
for file in site_files:
    df = pd.read_csv(file)
    dfs[file] = df.copy()
    print(file, "initial shape:", df.shape)

site_1_train_data.csv initial shape: (25081, 16)
site_2_train_data.csv initial shape: (25969, 16)
site_3_train_data.csv initial shape: (21913, 16)


In [ ]:
drop_cols = ['NO2_satellite', 'HCHO_satellite', 'ratio_satellite']
for file, df in dfs.items():
    for col in drop_cols:
        if col in df.columns:
            df.drop(columns=col, inplace=True)
    dfs[file] = df
    print(file, "after dropping satellite cols:", df.shape)

site_1_train_data.csv after dropping satellite cols: (25081, 13)
site_2_train_data.csv after dropping satellite cols: (25969, 13)
site_3_train_data.csv after dropping satellite cols: (21913, 13)


In [ ]:
for file, df in dfs.items():
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df = df.sort_values('datetime').reset_index(drop=True)
    dfs[file] = df
    print(file, "after datetime sort:", df.shape)

site_1_train_data.csv after datetime sort: (25081, 14)
site_2_train_data.csv after datetime sort: (25969, 14)
site_3_train_data.csv after datetime sort: (21913, 14)


In [ ]:
for file, df in dfs.items():
    df.interpolate(method='linear', limit_direction='both', inplace=True)
    df.dropna(inplace=True)
    dfs[file] = df
    print(file, "after interpolation & dropna:", df.shape)

site_1_train_data.csv after interpolation & dropna: (25081, 14)
site_2_train_data.csv after interpolation & dropna: (25969, 14)
site_3_train_data.csv after interpolation & dropna: (21913, 14)


In [ ]:
for file, df in dfs.items():
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['O3_diff'] = df['O3_target'] - df['O3_target'].shift(1)
    df['NO2_diff'] = df['NO2_target'] - df['NO2_target'].shift(1)
    df.fillna(0, inplace=True)
    dfs[file] = df

In [ ]:
for file, df in dfs.items():
    for pollutant in ['O3_target', 'NO2_target']:
        for lag in range(1, 73):
            df[f'{pollutant}_lag_{lag}'] = df[pollutant].shift(lag)
    df.dropna(inplace=True)
    dfs[file] = df

/tmp/ipython-input-1133038121.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{pollutant}_lag_{lag}'] = df[pollutant].shift(lag)
/tmp/ipython-input-1133038121.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{pollutant}_lag_{lag}'] = df[pollutant].shift(lag)
/tmp/ipython-input-1133038121.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To ge

In [ ]:
time_steps = 72

for file, df in dfs.items():
    input_features = [
        'O3_forecast', 'NO2_forecast', 'T_forecast', 'q_forecast',
        'u_forecast', 'v_forecast', 'w_forecast',
        'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
        'O3_diff', 'NO2_diff'
    ]
    lag_features = [col for col in df.columns if '_lag_' in col]
    input_features.extend(lag_features)
    target_features = ['O3_target', 'NO2_target']

    # Split train/test
    train_df, test_df = train_test_split(df, test_size=0.25, shuffle=False)
    train_dfs[file] = train_df
    test_dfs[file] = test_df

    # Input scaler (one per site)
    scaler_x = StandardScaler()
    X_train = scaler_x.fit_transform(train_df[input_features])
    X_test = scaler_x.transform(test_df[input_features])
    scaler_xs[file] = scaler_x

    # Create sequences for X
    Xs_train, Xs_test = [], []
    for i in range(len(X_train) - time_steps):
        Xs_train.append(X_train[i:(i + time_steps)])
    for i in range(len(X_test) - time_steps):
        Xs_test.append(X_test[i:(i + time_steps)])

    X_train_seq = np.array(Xs_train)
    X_test_seq = np.array(Xs_test)
    X_train_seqs[file] = X_train_seq
    X_test_seqs[file] = X_test_seq

    # Target scalers per pollutant (O3, NO2)
    for pollutant in target_features:
        scaler_y = StandardScaler()
        y_train_full = scaler_y.fit_transform(train_df[[pollutant]])
        y_test_full = scaler_y.transform(test_df[[pollutant]])

        ys_train, ys_test = [], []
        for i in range(len(y_train_full) - time_steps):
            ys_train.append(y_train_full[i + time_steps])
        for i in range(len(y_test_full) - time_steps):
            ys_test.append(y_test_full[i + time_steps])

        y_train_seq = np.array(ys_train)
        y_test_seq = np.array(ys_test)

        # Save the target-specific scaler
        scaler_ys[(file, pollutant)] = scaler_y
        y_train_seqs[(file, pollutant)] = y_train_seq
        y_test_seqs[(file, pollutant)] = y_test_seq

    print(file,
          "Train seq shape:", X_train_seq.shape,
          "Test seq shape:", X_test_seq.shape)


site_1_train_data.csv Train seq shape: (18684, 72, 157) Test seq shape: (6181, 72, 157)
site_2_train_data.csv Train seq shape: (19350, 72, 157) Test seq shape: (6403, 72, 157)
site_3_train_data.csv Train seq shape: (16308, 72, 157) Test seq shape: (5389, 72, 157)


In [ ]:
import os
import joblib

os.makedirs("saved_scalers", exist_ok=True)

for file, x_scaler in scaler_xs.items():
    site = file.replace("_train_data.csv", "")
    x_path = f"saved_scalers/{site}_X_scaler.pkl"
    joblib.dump(x_scaler, x_path)
    print("Saved X-scaler:", x_path)

for key, y_scaler in scaler_ys.items():
    if isinstance(key, tuple):
        file, pollutant = key
        site = file.replace("_train_data.csv", "")
        poll_short = pollutant.replace("_target", "")
        y_path = f"saved_scalers/{site}_{poll_short}_Y_scaler.pkl"
        joblib.dump(y_scaler, y_path)
        print(f"Saved Y-scaler for {site} - {poll_short}: {y_path}")


Saved X-scaler: saved_scalers/site_1_X_scaler.pkl
Saved X-scaler: saved_scalers/site_2_X_scaler.pkl
Saved X-scaler: saved_scalers/site_3_X_scaler.pkl
Saved Y-scaler for site_1 - O3: saved_scalers/site_1_O3_Y_scaler.pkl
Saved Y-scaler for site_1 - NO2: saved_scalers/site_1_NO2_Y_scaler.pkl
Saved Y-scaler for site_2 - O3: saved_scalers/site_2_O3_Y_scaler.pkl
Saved Y-scaler for site_2 - NO2: saved_scalers/site_2_NO2_Y_scaler.pkl
Saved Y-scaler for site_3 - O3: saved_scalers/site_3_O3_Y_scaler.pkl
Saved Y-scaler for site_3 - NO2: saved_scalers/site_3_NO2_Y_scaler.pkl


In [ ]:
import shutil

shutil.make_archive("saved_scalers_zip", 'zip', "saved_scalers")
print("Zipped → saved_scalers_zip.zip")


Zipped → saved_scalers_zip.zip


In [ ]:
for idx, file in enumerate(site_files, start=1):
    X_train_seq = X_train_seqs[file]
    X_test_seq = X_test_seqs[file]
    train_df = train_dfs[file]
    test_df = test_dfs[file]

    for pollutant in ['O3_target', 'NO2_target']:
        print(f"\n\n==================== {file} - {pollutant} ====================")

        scaler_y = StandardScaler()
        scaler_ys[(file, pollutant)] = scaler_y
        y_train_full = scaler_y.fit_transform(train_df[[pollutant]])
        y_test_full = scaler_y.transform(test_df[[pollutant]])

        ys_train = []
        for i in range(len(y_train_full) - time_steps):
            ys_train.append(y_train_full[i + time_steps])
        y_train_seq = np.array(ys_train)

        ys_test = []
        for i in range(len(y_test_full) - time_steps):
            ys_test.append(y_test_full[i + time_steps])
        y_test_seq = np.array(ys_test)

        y_train_seqs[(file, pollutant)] = y_train_seq
        y_test_seqs[(file, pollutant)] = y_test_seq

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
        ]

        inputs = Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))
        x = LSTM(128, return_sequences=True)(inputs)
        x = Dropout(0.2)(x)
        x = LSTM(64)(x)
        x = Dropout(0.2)(x)
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)
        outputs = Dense(1)(x)
        lstm_model = Model(inputs, outputs)
        lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        print("Training LSTM...")
        lstm_model.fit(X_train_seq, y_train_seq, validation_data=(X_test_seq, y_test_seq), epochs=100, batch_size=64, callbacks=callbacks, verbose=2)
        y_pred_lstm = lstm_model.predict(X_test_seq)

        inputs = Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))
        x = GRU(128, return_sequences=True)(inputs)
        x = Dropout(0.2)(x)
        x = GRU(64)(x)
        x = Dropout(0.2)(x)
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)
        outputs = Dense(1)(x)
        gru_model = Model(inputs, outputs)
        gru_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        print("Training GRU...")
        gru_model.fit(X_train_seq, y_train_seq, validation_data=(X_test_seq, y_test_seq), epochs=100, batch_size=64, callbacks=callbacks, verbose=2)
        y_pred_gru = gru_model.predict(X_test_seq)

        inputs = Input(shape=(X_train_seq.shape[1], X_train_seq.shape[2]))
        attn = MultiHeadAttention(num_heads=4, key_dim=64)(inputs, inputs)
        attn = Dropout(0.2)(attn)
        x = Add()([inputs, attn])
        x = LayerNormalization(epsilon=1e-6)(x)
        ffn = Dense(128, activation='relu')(x)
        ffn = Dropout(0.2)(ffn)
        ffn = Dense(X_train_seq.shape[2])(ffn)
        x = Add()([x, ffn])
        x = LayerNormalization(epsilon=1e-6)(x)
        x = GlobalAveragePooling1D()(x)
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)
        outputs = Dense(1)(x)
        trans_model = Model(inputs, outputs)
        trans_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        print("Training Transformer...")
        trans_model.fit(X_train_seq, y_train_seq, validation_data=(X_test_seq, y_test_seq), epochs=100, batch_size=64, callbacks=callbacks, verbose=2)
        y_pred_trans = trans_model.predict(X_test_seq)

        y_pred_ensemble_scaled = 0.4 * y_pred_gru + 0.4 * y_pred_lstm + 0.2 * y_pred_trans

        y_test_inv = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).reshape(-1)
        y_pred_inv = scaler_y.inverse_transform(y_pred_ensemble_scaled.reshape(-1, 1)).reshape(-1)

        results[(file, pollutant, 'Ensemble')] = {"y_true": y_test_inv, "y_pred": y_pred_inv}

        import os
        site_name = file.replace("_train_data.csv","")   # e.g. "site_1"
        poll_short = pollutant.replace("_target","")     # e.g. "O3" or "NO2"
        save_dir = os.path.join("saved_models", site_name)
        os.makedirs(save_dir, exist_ok=True)

        lstm_path = os.path.join(save_dir, f"{site_name}_{poll_short}_LSTM.keras")
        gru_path  = os.path.join(save_dir, f"{site_name}_{poll_short}_GRU.keras")
        trans_path= os.path.join(save_dir, f"{site_name}_{poll_short}_TRANS.keras")

        lstm_model.save(lstm_path)
        gru_model.save(gru_path)
        trans_model.save(trans_path)

        print(f"\n✔ Saved: {lstm_path}")
        print(f"✔ Saved: {gru_path}")
        print(f"✔ Saved: {trans_path}")
        # --- end save ---



==================== site_1_train_data.csv - O3_target ====================
Training LSTM...
Epoch 1/100
292/292 - 82s - 281ms/step - loss: 0.2979 - val_loss: 0.0893 - learning_rate: 1.0000e-03
Epoch 2/100
292/292 - 80s - 273ms/step - loss: 0.1738 - val_loss: 0.0793 - learning_rate: 1.0000e-03
Epoch 3/100


KeyboardInterrupt: 

In [ ]:
summary_rows = []

for (file, pollutant, model_name), data in results.items():
    y_true_inv = data["y_true"]
    y_pred_inv = data["y_pred"]
    site = file.split("_train_data.csv")[0]

    rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
    mae = mean_absolute_error(y_true_inv, y_pred_inv)
    r2 = r2_score(y_true_inv, y_pred_inv)
    obs_mean = y_true_inv.mean()
    numerator = np.sum((y_pred_inv - y_true_inv) ** 2)
    denominator = np.sum((np.abs(y_pred_inv - y_true_inv) + np.abs(y_true_inv - obs_mean)) ** 2)
    ria = 1 - (numerator / denominator) if denominator != 0 else np.nan

    summary_rows.append({
        "Site": site,
        "Model": model_name,
        "Target": pollutant,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2,
        "RIA": ria
    })

summary_df = pd.DataFrame(summary_rows)
print("\n PERFORMANCE SUMMARY (All Sites & Models) ")
print(summary_df.to_string(index=False))

avg_summary = summary_df.groupby(["Site", "Model"])[["RMSE","MAE","R2","RIA"]].mean().reset_index()
print("\nAVERAGE PERFORMANCE PER SITE")
print(avg_summary.to_string(index=False))

best_per_site = avg_summary.loc[avg_summary.groupby("Site")["R2"].idxmax()]
print("\n BEST MODEL PER SITE (By R²) ")
print(best_per_site.to_string(index=False))

overall_best = avg_summary.groupby("Model")[["R2","RIA"]].mean().reset_index().sort_values(by="R2", ascending=False).head(1)
print("\n OVERALL BEST MODEL")
print(overall_best.to_string(index=False))

In [ ]:
import shutil

shutil.make_archive("saved_models", 'zip', "saved_models")

In [ ]:
from google.colab import files
files.download("saved_models.zip")
